In [1]:
from tensorflow.keras.datasets import cifar10

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [6]:
X_train.shape

(50000, 32, 32, 3)

In [7]:
from pathlib import Path

In [18]:
p = Path('..')

In [19]:
list(p.iterdir())

[WindowsPath('../.git'),
 WindowsPath('../.gitignore'),
 WindowsPath('../.ipynb_checkpoints'),
 WindowsPath('../automobile_consulting_company_predicting_cars_prices'),
 WindowsPath('../beginner_python_projects'),
 WindowsPath('../Business Case Study - Absenteeism'),
 WindowsPath('../Business Case Study - AudioBook App'),
 WindowsPath('../cell_images'),
 WindowsPath('../cifar-10'),
 WindowsPath('../Credit Card Fraud Detection'),
 WindowsPath('../Data_Science_BEST_practices'),
 WindowsPath('../Directing Customers to Subscription Through App Behavior Analysis'),
 WindowsPath('../Disaster_Or_Not_Kaggle_Competition'),
 WindowsPath('../Fashion_class_classification'),
 WindowsPath('../House_price_prediction_Kaggle_competition'),
 WindowsPath('../IBM_HR_Analytics_Employee_And_Performance'),
 WindowsPath('../loan_analysis_problem_statement'),
 WindowsPath('../Machine_Learning_Algorithms_Tutorials'),
 WindowsPath('../Machine_learning_with_text'),
 WindowsPath('../Minimizing Churn Rate Through An

In [20]:
[x for x in p.iterdir() if x.is_dir()]

[WindowsPath('../.git'),
 WindowsPath('../.ipynb_checkpoints'),
 WindowsPath('../automobile_consulting_company_predicting_cars_prices'),
 WindowsPath('../beginner_python_projects'),
 WindowsPath('../Business Case Study - Absenteeism'),
 WindowsPath('../Business Case Study - AudioBook App'),
 WindowsPath('../cell_images'),
 WindowsPath('../cifar-10'),
 WindowsPath('../Credit Card Fraud Detection'),
 WindowsPath('../Data_Science_BEST_practices'),
 WindowsPath('../Directing Customers to Subscription Through App Behavior Analysis'),
 WindowsPath('../Disaster_Or_Not_Kaggle_Competition'),
 WindowsPath('../Fashion_class_classification'),
 WindowsPath('../House_price_prediction_Kaggle_competition'),
 WindowsPath('../IBM_HR_Analytics_Employee_And_Performance'),
 WindowsPath('../loan_analysis_problem_statement'),
 WindowsPath('../Machine_Learning_Algorithms_Tutorials'),
 WindowsPath('../Machine_learning_with_text'),
 WindowsPath('../Minimizing Churn Rate Through Analysis of Financial Habits'),
 

In [21]:
p.absolute()

WindowsPath('C:/Users/fares/OneDrive/Bureau/DS_GitHub_Projects/cifar-10/..')

In [26]:
list(p.glob('*'))

[WindowsPath('../.git'),
 WindowsPath('../.gitignore'),
 WindowsPath('../.ipynb_checkpoints'),
 WindowsPath('../automobile_consulting_company_predicting_cars_prices'),
 WindowsPath('../beginner_python_projects'),
 WindowsPath('../Business Case Study - Absenteeism'),
 WindowsPath('../Business Case Study - AudioBook App'),
 WindowsPath('../cell_images'),
 WindowsPath('../cifar-10'),
 WindowsPath('../Credit Card Fraud Detection'),
 WindowsPath('../Data_Science_BEST_practices'),
 WindowsPath('../Directing Customers to Subscription Through App Behavior Analysis'),
 WindowsPath('../Disaster_Or_Not_Kaggle_Competition'),
 WindowsPath('../Fashion_class_classification'),
 WindowsPath('../House_price_prediction_Kaggle_competition'),
 WindowsPath('../IBM_HR_Analytics_Employee_And_Performance'),
 WindowsPath('../loan_analysis_problem_statement'),
 WindowsPath('../Machine_Learning_Algorithms_Tutorials'),
 WindowsPath('../Machine_learning_with_text'),
 WindowsPath('../Minimizing Churn Rate Through An